In [1]:
# Importing necessary libraries 
import os
import json
import pandas as pd
import numpy as np
import glob
from IPython import get_ipython


# For data cleaning and visualization
import pandas as pd
import numpy as np
from numpy import array
from datetime import date
from datetime import datetime, timedelta
import pandas_datareader.data as web


# For saving the model
import pickle

# For model Evaluation
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import math

import re
import nltk
from nltk.corpus import stopwords

In [115]:
company = 'NIO'
scores_name = "FinalData/vadar_scores_"+str(company)+".csv"
mean_scores_name = "FinalData/vadar_mean_compound_scores_"+str(company)+".csv"

In [116]:
scores_df = pd.read_csv(scores_name, index_col=0)
mean_scores = pd.read_csv(mean_scores_name, index_col=0)

In [117]:
scores_df

,headline,date,neg,neu,pos,compound
0,Get SMS ing U S tech execs urge Web starved ...,2009-04-22,0.310,0.690,0.000,-0.5574
1,CORRECTED Little known cloud startups attrac...,2009-05-06,0.000,0.522,0.478,0.6705
2,US STOCKS Wall St climbs on tech shares earni...,2009-07-16,0.000,0.584,0.416,0.6908
3,US STOCKS Wall St flat as data offsets profit ...,2009-07-16,0.000,0.556,0.444,0.7506
4,US STOCKS Wall St opens flat as data offsets e...,2009-07-17,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...
5150,NVIDIA Officially Launches Google Stadia Compe...,2020-02-05,0.000,1.000,0.000,0.0000
5149,Intel bets on smart buildings in Israel to att...,2020-02-05,0.000,0.500,0.500,0.7906
5148,3 In Demand Software Stocks Worth Considering ...,2020-02-05,0.132,0.702,0.167,0.1027
5158,Alphabet Falls on Revenue Miss but Wall Stree...,2020-02-05,0.080,0.612,0.309,0.6956


In [118]:
mean_scores

,mean_compound,date
0,-0.557400,2009-04-22
1,0.670500,2009-05-06
2,0.720700,2009-07-16
3,0.000000,2009-07-17
4,0.000000,2009-08-09
...,...,...
1646,-0.318200,2020-02-01
1647,0.018391,2020-02-02
1648,0.118797,2020-02-03
1649,0.069568,2020-02-04


In [119]:
# Helper Functions 
def getTickerData(ticker, start, end):
  # exception handling for unexpected errors
  try:
    df = web.DataReader(ticker, 'yahoo', start, end)
    # convert the web data to csv format
    df.to_csv('{}.csv'.format(ticker), index=True)
  except:
    print("Error in getting Data...")
def get_data(start_date, end_date, company, stkMarket=""):
  # Ticker Information  
  # Ticker Examples = ['MMM',  'ABT',  'AAPL',  'AMAT',  'APTV',  'ADM',  'ARNC',  'AMZN', 'GOOG']   
  # Stock data retrieval using yahoo
  ticker = company
  stockMarket = stkMarket
  if stockMarket:
    ticker = ticker + "." + stockMarket
    print("Ticker = ",ticker)
  print(f"Collecting the required data for {ticker} from dates: {start_date} to {end_date}...")
  getTickerData(ticker, start_date, end_date)
  df = web.DataReader(ticker, 'yahoo', start_date, end_date)
  df = pd.read_csv('{}.csv'.format(ticker))
  return df

In [120]:
# setting start and end date for stock data retrieval
start_date = date(2008,1,1)
end_date = date(2021,1,1)

# fetching data using ticker
sdf = get_data(start_date,end_date,company)
sdf

,Date,High,Low,Open,Close,Volume,Adj Close
0,2018-09-12,6.930000,5.350000,6.000000,6.600000,66849000,6.600000
1,2018-09-13,12.690000,6.520000,6.620000,11.600000,158346500,11.600000
2,2018-09-14,13.800000,9.220000,12.660000,9.900000,172473600,9.900000
3,2018-09-17,9.750000,8.500000,9.610000,8.500000,56323900,8.500000
4,2018-09-18,9.100000,7.670000,8.730000,7.680000,41827600,7.680000
...,...,...,...,...,...,...,...
576,2020-12-24,47.130001,45.349998,46.880001,45.770000,60582500,45.770000
577,2020-12-28,46.849998,43.669998,46.669998,44.060001,105078000,44.060001
578,2020-12-29,46.279999,42.230000,43.700001,46.139999,120689900,46.139999
579,2020-12-30,48.380001,45.720001,46.500000,48.380001,116691700,48.380001


In [121]:
sdf = sdf.dropna()
sdf = sdf.reset_index(drop=True)
sdf

,Date,High,Low,Open,Close,Volume,Adj Close
0,2018-09-12,6.930000,5.350000,6.000000,6.600000,66849000,6.600000
1,2018-09-13,12.690000,6.520000,6.620000,11.600000,158346500,11.600000
2,2018-09-14,13.800000,9.220000,12.660000,9.900000,172473600,9.900000
3,2018-09-17,9.750000,8.500000,9.610000,8.500000,56323900,8.500000
4,2018-09-18,9.100000,7.670000,8.730000,7.680000,41827600,7.680000
...,...,...,...,...,...,...,...
576,2020-12-24,47.130001,45.349998,46.880001,45.770000,60582500,45.770000
577,2020-12-28,46.849998,43.669998,46.669998,44.060001,105078000,44.060001
578,2020-12-29,46.279999,42.230000,43.700001,46.139999,120689900,46.139999
579,2020-12-30,48.380001,45.720001,46.500000,48.380001,116691700,48.380001


In [122]:
sdf['Date'] = pd.to_datetime(sdf['Date'])
sdf = sdf.sort_values(by=['Date'], ascending=True)
sdf = sdf.reset_index(drop=True)
sdf

,Date,High,Low,Open,Close,Volume,Adj Close
0,2018-09-12,6.930000,5.350000,6.000000,6.600000,66849000,6.600000
1,2018-09-13,12.690000,6.520000,6.620000,11.600000,158346500,11.600000
2,2018-09-14,13.800000,9.220000,12.660000,9.900000,172473600,9.900000
3,2018-09-17,9.750000,8.500000,9.610000,8.500000,56323900,8.500000
4,2018-09-18,9.100000,7.670000,8.730000,7.680000,41827600,7.680000
...,...,...,...,...,...,...,...
576,2020-12-24,47.130001,45.349998,46.880001,45.770000,60582500,45.770000
577,2020-12-28,46.849998,43.669998,46.669998,44.060001,105078000,44.060001
578,2020-12-29,46.279999,42.230000,43.700001,46.139999,120689900,46.139999
579,2020-12-30,48.380001,45.720001,46.500000,48.380001,116691700,48.380001


In [123]:
mean_scores

,mean_compound,date
0,-0.557400,2009-04-22
1,0.670500,2009-05-06
2,0.720700,2009-07-16
3,0.000000,2009-07-17
4,0.000000,2009-08-09
...,...,...
1646,-0.318200,2020-02-01
1647,0.018391,2020-02-02
1648,0.118797,2020-02-03
1649,0.069568,2020-02-04


In [124]:
mean_scores['date'] = pd.to_datetime(mean_scores['date'])
mean_scores = mean_scores.sort_values(by=['date'], ascending=True)
mean_scores = mean_scores.reset_index(drop=True)
mean_scores

,mean_compound,date
0,-0.557400,2009-04-22
1,0.670500,2009-05-06
2,0.720700,2009-07-16
3,0.000000,2009-07-17
4,0.000000,2009-08-09
...,...,...
1646,-0.318200,2020-02-01
1647,0.018391,2020-02-02
1648,0.118797,2020-02-03
1649,0.069568,2020-02-04


In [125]:
sdf.shape, mean_scores.shape

((581, 7), (1651, 2))

In [126]:
sdf = sdf.drop(columns=['High', 'Low','Open','Close','Volume'])
sdf

,Date,Adj Close
0,2018-09-12,6.600000
1,2018-09-13,11.600000
2,2018-09-14,9.900000
3,2018-09-17,8.500000
4,2018-09-18,7.680000
...,...,...
576,2020-12-24,45.770000
577,2020-12-28,44.060001
578,2020-12-29,46.139999
579,2020-12-30,48.380001


In [127]:
sdf['date'] = sdf['Date']
sdf.drop(columns=['Date'],inplace=True)
sdf

,Adj Close,date
0,6.600000,2018-09-12
1,11.600000,2018-09-13
2,9.900000,2018-09-14
3,8.500000,2018-09-17
4,7.680000,2018-09-18
...,...,...
576,45.770000,2020-12-24
577,44.060001,2020-12-28
578,46.139999,2020-12-29
579,48.380001,2020-12-30


In [128]:
merged_inner = pd.merge(left=mean_scores, right=sdf, left_on='date', right_on='date')
merged_inner

,mean_compound,date,Adj Close
0,0.102367,2018-09-12,6.60
1,0.263350,2018-09-13,11.60
2,-0.372300,2018-09-14,9.90
3,0.000000,2018-09-17,8.50
4,0.000000,2018-09-19,8.50
...,...,...,...
336,0.027358,2020-01-30,4.08
337,0.260790,2020-01-31,3.78
338,0.118797,2020-02-03,4.06
339,0.069568,2020-02-04,4.51


In [129]:
merged_inner = merged_inner.rename(columns = {'Adj Close': 'adj_close'})
df_final = merged_inner[['date','adj_close','mean_compound']]
df_final

,date,adj_close,mean_compound
0,2018-09-12,6.60,0.102367
1,2018-09-13,11.60,0.263350
2,2018-09-14,9.90,-0.372300
3,2018-09-17,8.50,0.000000
4,2018-09-19,8.50,0.000000
...,...,...,...
336,2020-01-30,4.08,0.027358
337,2020-01-31,3.78,0.260790
338,2020-02-03,4.06,0.118797
339,2020-02-04,4.51,0.069568


In [130]:
df_name = "FinalData/CS_stock_and_news_"+company+".csv"
df_final.to_csv(df_name)